In [5]:
from datasets import load_dataset
import datasets
ds = load_dataset("openai/gsm8k", "main")
train: datasets.Dataset = ds["train"]
prompt_templ = """A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: {0}
Assistant: <think>"""
print(prompt_templ)

A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: {0}
Assistant: <think>


In [7]:
from cs336_alignment.math_baseline import extract_answer
prompts = []
ground_truths = []
for t, data in enumerate(train):
    question = data["question"]
    answer_text = data["answer"]
    answer = extract_answer(answer_text)
    assert answer is not None, f"Could not extract answer from: {answer_text}"
    full_prompt = prompt_templ.format(question)
    prompts.append(full_prompt)
    ground_truths.append(answer)

In [19]:
import random
index = random.randint(0, len(prompts) - 1)
print("Question:")
print(train[index]["question"])
print("-" * 20)
print("Ground truth answer:")
print(ground_truths[index])
print("-" * 20)
print("Answer Text:")
print(train[index]["answer"])

Question:
Rica's group won in a dance competition. She got 3/8 of the prize money. From Rica's prize money, she spent 1/5 of it and is now left with $300. How much was the prize money that her group won?
--------------------
Ground truth answer:
1000
--------------------
Answer Text:
Rica is left with 1 - 1/5 = 4/5 of her prize money which is equal to $300.
Since 4/5 is worth $300, then 1/5 is worth $300/4 = $75.
So, Rica got $75 x 5 = $375 from their prize money which is 3/8 of the total prize.
Since 3/8 is equal to $375, then 1/8 is worth $375/3 = $125.
So, the total prize money is $125 x 8 = $<<125*8=1000>>1000.
#### 1000


In [2]:
import pickle
with open("../data/math_baseline_eval_results.pkl", "rb") as f:
    results = pickle.load(f)

In [9]:
import random
prompts = results["prompts"]
completions = results["completions"]
rewards = results["rewards"]
ground_truths = results["ground_truths"]

index = random.randint(0, len(prompts) - 1)
print("Prompt:")
print(prompts[index])
print("-" * 20)
print("Completion:")
print(completions[index])
print("-" * 20)
print("Ground Truth:")
print(ground_truths[index])
print("-" * 20)
print("Reward:")
print(rewards[index])

Prompt:
A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: A bulk warehouse is offering 48 cans of sparkling water for $12.00 a case.  The local grocery store is offering the same sparkling water for $6.00 and it only has 12 cans.  How much more expensive, per can, in cents, is this deal at the grocery store?
Assistant: <think>
--------------------
Completion:
 This question can be approached by calculating the price per can at each store and comparing
--------------------
Ground Truth:
25
--------------------
Reward:
{'format_reward': 0.0, 'answer_reward': 0.0, 'reward': 0.0}
